# Exploration

Reformatting Q Report data.

In [1]:
import pandas as pd
from tqdm import tqdm

In [ ]:
pd.read_pickle('data/evaluations/evaluations.pkl').head()

## Reformatting data

In [2]:
renaming = {
    "Course Response Rate": "courseResponseRate",
    "Course General Questions": "courseGeneralQuestions",
    "General Instructor Questions": "generalInstructorQuestions",
    "On average, how many hours per week did you spend on coursework outside of class? Enter a whole number between 0 and 168.": "hoursPerWeek",
    "How strongly would you recommend this course to your peers?": "recommendation",
    "What was/were your reason(s) for enrolling in this course? (Please check all that apply)": "reasons",
    "What would you like to tell future students about this class? (Your response to this question may be published anonymously.)": "comments",
    "What would you like to tell future students about this class?": "commentsAlt",
}


dfs = []
for i in range(1, 16):
    path = f"data/evaluations/qreports-2023-04-14/batch-{i}.json"
    dfs.append(pd.read_json(path))

df = pd.concat(dfs).reset_index(drop=True).rename(columns=renaming)

In [3]:
assert not (df["comments"].notna() & df["commentsAlt"].notna()).any()
df["comments"] = df["comments"].fillna(df["commentsAlt"])
df = df.drop(columns=["commentsAlt"])

In [4]:
dict_columns = [
    "courseResponseRate",
    "courseGeneralQuestions",
    "generalInstructorQuestions",
    "hoursPerWeek",
    "recommendation",
    "reasons",
]

expanded_dfs = []
for column in tqdm(dict_columns):
    expanded_dfs.append(pd.json_normalize(df[column], sep="_").add_prefix(f"{column}_"))

df = pd.concat([df.drop(columns=dict_columns), *expanded_dfs], axis=1)

100%|██████████| 6/6 [00:06<00:00,  1.16s/it]


In [5]:
def get_tuple(column):
    parts = column.split("_")
    if len(parts) == 1:
        return ("meta", column, "")
    elif len(parts) == 2:
        return (parts[0], parts[1], "")
    else:
        return tuple(parts)


# create a multi-index for the columns
df.columns = pd.MultiIndex.from_tuples(
    [get_tuple(column) for column in df.columns],
)
df

meta                \
                                                    url  year  season   
                                                                        
0     https://harvard.bluera.com/harvard/rpv-eng.asp...  2022  Spring   
1     https://harvard.bluera.com/harvard/rpv-eng.asp...  2022  Spring   
2     https://harvard.bluera.com/harvard/rpv-eng.asp...  2022  Spring   
3     https://harvard.bluera.com/harvard/rpv-eng.asp...  2022  Spring   
4     https://harvard.bluera.com/harvard/rpv-eng.asp...  2022  Spring   
...                                                 ...   ...     ...   
6914  https://harvard.bluera.com/harvard/rpv-eng.asp...  2019    Fall   
6915  https://harvard.bluera.com/harvard/rpv-eng.asp...  2019    Fall   
6916  https://harvard.bluera.com/harvard/rpv-eng.asp...  2019    Fall   
6917  https://harvard.bluera.com/harvard/rpv-eng.asp...  2019    Fall   
6918  https://harvard.bluera.com/harvard/rpv-eng.asp...  2019    Fall   

                                               \
                courseName     instructorName   
                                                
0                 ARTS 27R       Davone Tines   
1                 ARTS 27R      Isaac Winokur   
2               AFRAMER 10        Henry Gates   
3               AFRAMER 10   Evelynn Hammonds   
4             AFRAMER 109Y         Vivek Bald   
...                    ...                ...   
6914         WOMGEN 1210FT  Afsaneh Najmabadi   
6915           WOMGEN 1217        Nicole Noll   
6916           WOMGEN 1247    Michael Bronski   
6917           WOMGEN 1273        Robert Reid   
6918  EMR 131, WOMGEN 1283     Caroline Light   

                                                        courseResponseRate  \
                                               comments          responded   
                                                                             
0     [This is one of the best classes I've ever tak...                  5   
1     [This is one of the best classes I've ever tak...                  5   
2     [This is a great intro course. The materials i...                 48   
3     [This is a great intro course. The materials i...                 48   
4     [It's very interesting and lowkey until the la...                  5   
...                                                 ...                ...   
6914  [This class was super interesting and a great ...                  8   
6915  [TAKE THIS CLASS!!! It is a mindblowing introd...                 14   
6916  [if you are really really interested in music,...                 18   
6917  [there's A LOT of reading, but it's all worthw...                  8   
6918  [Caroline Light is the best seminar leader I h...                 14   

                   courseGeneralQuestions                     ...  \
     invited Evaluate the course overall.                     ...   
                                    count              votes  ...   
0          8                          3.0    [3, 0, 0, 0, 0]  ...   
1          8                          3.0    [3, 0, 0, 0, 0]  ...   
2         62                         40.0  [18, 13, 7, 2, 0]  ...   
3         62                         40.0  [18, 13, 7, 2, 0]  ...   
4         12                          4.0    [1, 3, 0, 0, 0]  ...   
...      ...                          ...                ...  ...   
6914      12                          8.0    [5, 2, 1, 0, 0]  ...   
6915      16                         14.0   [13, 1, 0, 0, 0]  ...   
6916      20                         17.0   [11, 3, 3, 0, 0]  ...   
6917      10                          8.0    [8, 0, 0, 0, 0]  ...   
6918      17                         14.0   [14, 0, 0, 0, 0]  ...   

     recommendation  reasons                                               \
              stdev elective concentration secondary gened expos language   
                                                                            
0              0.00      3.0           0.0       0

In [6]:
df.to_pickle("data/evaluations/evaluations.pkl")